In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.integrate import solve_ivp 

In [6]:
def f(z,V,k,m,a,R):                                     # def f(r,V,k,m,a,c,α):
#     Vt = 1.0;
#     R = 1.0;
    #################################
    # Γt
    G = (1-4*m**2+3*z**2+4*a**2 * m**2 -8*m**2 * z**2 + 3*z**4 - 4*m**2 * z**4 + z**6)
    ###
    Γt_tr = (m*(z+2*z**3+z**5+2*m-a*m*a**2+4*m*z**2+4*m*a**2 * z**2 + 2*m*z**4))/ (z**2 * G)
    Γt_tz = (m*z*(z+2*z**3+z**5+2*m-6*m*a**2+4*m*z**2+2*m*z**4))/(z**2 * G)
    Γt_rt = Γt_tr
    Γt_rϕ = (-m*a*(z+4*m))/(G)
    Γt_ϕr = Γt_rϕ
    Γt_ϕz = (-m*a*z*(3*z+4*m))/(G)
    Γt_zt = Γt_tz
    Γt_zϕ = Γt_ϕz
    #################################
    # Γr
    H = (z+2*m)*z**2
    ####
    Γr_tt = m/H
    Γr_tϕ = (2*m*a)/H
    Γr_rr = -m/H
    Γr_rz = -m*z/H
    Γr_ϕt = 2*m*a/H
    Γr_ϕϕ = -1*(z**3+m+2*m*z**2)/H
    Γr_zr = (-m*z)/H
    Γr_zz = m/H
#######################
    # Γϕ
    Γϕ_tz = ((a*m*z)*(-4*m+3*z))/(G)
    Γϕ_zϕ = (-1*(m*z)*(z+2*z**3+z**5-2*m+6*m*a**2-4*m*z**2-2*m*z**4))/((1+z**2)*G)
    Γϕ_ϕz = -Γϕ_zϕ    
#######################
    # Γz
    L = (1+z**2)*(z+2*m)
    Γz_tt = (m*z)/(L)
    Γz_tϕ = (-1*(3*m*a*z)/(L*(1+z**2)))
    Γz_rr = (m*z)/(L)
    Γz_rz = (-m*R)/(L)
    Γz_ϕt = (-3*m*a*z*R**2)/(L*(1+z**2))
    Γz_ϕϕ = (m*z)/(L*(1+z**2))
    Γz_zr = -m/L
    Γz_zz = (-m*z)/((1+z**2)*(np.sqrt(z)+2*m))
#######################################
#     γ = 1/(np.sqrt(1 - (Vr**2 + Vϕ**2)));
    
    
#     Jr = 1.0;
#     Jϕ = 1.0;
#     Jz = 1.0;
    Uϕ = 1.0;
    U0 = 1.0;
    
    
    
    
    
#     Vr,Vϕ,Bϕ = V;
    
#     U0 = γ;  
#     Ur = γ*Vr;    
#     Uϕ = γ*Vϕ;
####################################       # dVz_dz computation 
    A = (z**1.5 + k*z**2.5);
    dVz_dz = (Γt_ϕz + Γt_zϕ)*(V[0]*V[1]) - \
    (2*V[2]*Jr*Uϕ*U0)/(A*U0**2) - (V[0]*(1.5*z**0.5 - 2.5*k*z**1.5))/(A)
####################################       # dVϕ_dz computation
#     Bz = (2.5*K*(1+2*m/z)*z**1.5)/(Jϕ) 
#     ((A*U0**2)/(Jϕ))*(Γr_tt + 2*dVϕ_dz*Γr_tϕ + dVϕ_dz*dVϕ_dz*Γr_ϕϕ + dVz_dz*dVz_dz*Γr_zz) 

    dVϕ_dz = -2*(Γϕ_tz - Γt_tz*V[1]) - V[1]*(Γϕ_zϕ - Γt_zϕ*V[1])- \
    V[1]*(Γϕ_ϕz - Γt_ϕz*V[1]) + (V[2]*Jr)/(V[0]*A*U0**2)
###################################           # Bz computation 
    Bz = (2.5*k*(1+2*m/z)*z**1.5)/(Jϕ) + \
    ((A*U0**2)/(Jϕ))*(Γr_tt + 2*dVϕ_dz*Γr_tϕ + dVϕ_dz*dVϕ_dz*Γr_ϕϕ + dVz_dz*dVz_dz*Γr_zz)
##################################            # Bϕ computation 
    D = ((-1.0)/(4*np.pi))*(2*z*(z**2 - 2*m)) + ((-1.0)/(4*np.pi))* \
    (((m*z)*(z**5 - 2*m*z**4 + 2*z**3 - 4*m*z**2 + z + 6*m*a**2 - 2*m)/ \
      (z**6 -4*m**2 * z**4 + 3*z**4 - 8*m**2 * z**2 + 3*z**2+4*a**2 * m**2 - 4*m**2 + 1 ))+\
     (m / z*(z+2*m)) +  ((m*z)/((1+z**2)*(np.sqrt(z)+2*m))))*(z**2 / (z**2 + 2*m)**2)

    Bϕ = (1.0/(D*U0*Uϕ*Bz))*((((U0**2 * A *dVz_dz*dVϕ_dz*(Γt_ϕz + Γt_zϕ))/(2.0)) - \
                          ((U0**2 *dVz_dz*(1.5*z**0.5 - 2.5*k*z**1.5))/(2.0))) + \
                         ((((1 + 2*m/z)*2.5*k*z**1.5)/(2*dVz_dz)) + \
                          (((A*U0**2)/(2*dVz_dz))*\
                           (Γz_tt + 2*dVϕ_dz*Γz_tϕ-2*dVz_dz*dVz_dz*Γt_tz + \
                        dVϕ_dz*dVϕ_dz*Γz_ϕϕ - Γt_zϕ*dVz_dz*dVz_dz*dVϕ_dz - \
                           Γt_ϕz*dVz_dz*dVz_dz*dVϕ_dz + dVz_dz*dVz_dz*Γz_zz)))) 
    
    dVdz = [dVz_dz, dVϕ_dz, Bz,Bϕ]
    return dVdz

In [7]:
# def f(z,V,k,m,a,R):
f(1,[0.12,0.918,1e4],0.364,1e3,0.2,1)

NameError: name 'Jr' is not defined

In [ ]:
# f(r,V,k,m,a,c,α)
V = solve_ivp(fun=f, t_span=[3,10], y0=[0.12,0.918,1e2,1e4],
              args=(0.364,1e3,0.2,1), dense_output=False)  
